# Weather forecast and actual weather

Using the API, we try to compare the weather forecast of last week with the actual weather occuring on that date.
For this purpose, we use the OpenWeather API to retrieve all necessary information.

In [1]:
%pip install kafka-python
%run Functions.ipynb import *

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


Exception: File `'Functions.ipynb.py'` not found.

### Setup
Import required libraries and set up global variables.

In [ ]:
import requests
import datetime
import logging as log
from kafka import KafkaProducer
from kafka.errors import KafkaError

API_KEY = 'fac9ad607136e9c0f991980b5cd58ed7'

### Data request
Request weather forecast data from the OpenWeather API for Mannheim, Germany (located at 49.4875, 8.4660).

In [ ]:
url = 'https://api.openweathermap.org/data/2.5/onecall'
response = requests.get(url, params={
    'lat': 49.4875,
    'lon': 8.4660,
    'units': 'metric',
    'exclude': 'current,minutely,hourly,alerts',
    'appid': API_KEY
})

response.json()

{'lat': 49.4875,
 'lon': 8.466,
 'timezone': 'Europe/Berlin',
 'timezone_offset': 3600,
 'daily': [{'dt': 1636455600,
   'sunrise': 1636439260,
   'sunset': 1636473141,
   'moonrise': 1636459380,
   'moonset': 1636487340,
   'moon_phase': 0.18,
   'temp': {'day': 10.32,
    'min': 4.93,
    'max': 10.94,
    'night': 6,
    'eve': 9.71,
    'morn': 5.19},
   'feels_like': {'day': 8.79, 'night': 6, 'eve': 8.68, 'morn': 5.19},
   'pressure': 1028,
   'humidity': 53,
   'dew_point': 1.17,
   'wind_speed': 3.6,
   'wind_deg': 101,
   'wind_gust': 6.51,
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': 65,
   'pop': 0,
   'uvi': 1.16},
  {'dt': 1636542000,
   'sunrise': 1636525758,
   'sunset': 1636559456,
   'moonrise': 1636548420,
   'moonset': 1636578360,
   'moon_phase': 0.21,
   'temp': {'day': 7.73,
    'min': 3.94,
    'max': 8.79,
    'night': 5.56,
    'eve': 7.02,
    'morn': 4.16},
   'feels_like': {'day': 7.

### Data filtering
Filter the data received from the API. For the forecast for today and the forecast one week from now, find the following data:
- Date
- Temperature (average temperature throughout the day, Celsius)
- Rain (precipitation volume, mm)
- Snow (snow volume, mm)
- Clouds (cloudiness, %)
- Wind speed (average wind speed throughout the day, m/s)

In [ ]:
json = response.json()
daily_forecasts = json['daily']

forecasts = {
    day: {
        'date': datetime.datetime.fromtimestamp(daily_forecasts[index]['dt']),
        'temp': daily_forecasts[index]['temp']['day'],
        'rain': daily_forecasts[index]['rain'] if 'rain' in daily_forecasts[index] else 0,
        'snow': daily_forecasts[index]['snow'] if 'snow' in daily_forecasts[index] else 0,
        'clouds': daily_forecasts[index]['clouds'],
        'wind_speed': daily_forecasts[index]['wind_speed']
    }
    for day, index in [('today', 0), ('7 days', -1)]
}

forecasts

{'today': {'date': datetime.datetime(2021, 11, 9, 11, 0),
  'temp': 10.32,
  'rain': 0,
  'snow': 0,
  'clouds': 65,
  'wind_speed': 3.6},
 '7 days': {'date': datetime.datetime(2021, 11, 16, 11, 0),
  'temp': 9.23,
  'rain': 0,
  'snow': 0,
  'clouds': 39,
  'wind_speed': 1.9}}

### Data publication
Send the acquired data to the Kafka platform.

In [ ]:
producer = KafkaProducer(bootstrap_servers='localhost:0000')
try:
    producer.send('forecast', forecasts)
except KafkaError:
    log.exception('Send error')
    pass

NoBrokersAvailable: NoBrokersAvailable

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa73dd04-f36c-4056-b6c3-896407612102' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>